In [1]:
import numpy as np
import matplotlib as plt
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '../rainforest')
from paths import DATA_FOLDER
from data import get_class_data
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from keras.models import Model

Using Theano backend.


In [2]:
train = get_class_data(train=True, label='mines')
validation = get_class_data(train=False, label='mines')

### Info

###### Validation:
    artisial mine: 95, conventional mine: 26, total: 12144
###### Train:
    artisial mine: 244, conventional mine: 74, total: 28335
    
    2 images in train set with both labels
    1 image in val set with both labels

In [3]:
# data generator
def data_generator(data, label, balanced=True):
    '''
    Input:
        data: pandas object containing the data 
        label: label of data you want to generate images from
        balance: class data
    '''
    while True:
        yield None

In [4]:
# augmentation generator
def augmentation_generator(data_gen):
    '''
    Input:
        data_gen: a data generator that generates batches of data that need to be augmented
    '''
    while True:
        yield None

In [5]:
def model():
    model = VGG16(include_top=False, weights='imagenet', input_shape=(3,256,256))
    
    x = model.output
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)

    model = Model(inputs=model.input, outputs=predictions)
    
    for layer in model.layers:
        layer.trainable = False
    return model

In [6]:
a = model()
a.compile(optimizer='adam', loss='categorical_crossentropy')